In [1]:
%pip install opencv-python
%pip install ultralytics
%pip install langchain
%pip install twilio
%pip install chroma-hnswlib --prefer-binary

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain langchain_community sentence_transformers langchain_groq twilio markdown

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install crewai

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import numpy as np

In [5]:
path = 'Dataset.csv'
df = pd.read_csv(path)
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [6]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [7]:
for i in range(0, len(df)):
    title = df['المخالفة'].iloc[i]
    content = df['الغرامة'].iloc[i]

    markdown_content = f"{title} {content}\n\n"

    if i == 0:
      print(markdown_content)

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. الغرامة المالية 100 - 150 ريال




In [8]:
markdown_texts = []
for filename in os.listdir(directory):
    if filename.endswith(".md"):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            markdown_content = file.read()  
            markdown_texts.append(markdown_content)

In [9]:
max_length = df['المخالفة'].str.len().max() + df['الغرامة'].str.len().max() + 1
text_splitter = RecursiveCharacterTextSplitter(chunk_size=max_length,chunk_overlap=20)
documents=text_splitter.create_documents(markdown_texts)

In [10]:
documents[0]

Document(page_content='قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. الغرامة المالية 100 - 150 ريال')

In [11]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(documents,embedding_function,persist_directory="./chroma_db")

C:\Users\saad9\AppData\Local\Temp\ipykernel_26704\2084324871.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\saad9\.conda\envs\Python3-11-9\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\saad9\.conda\envs\Python3-11-9\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by defaul

## YOLO MODEL

In [12]:
import cv2
import re
import numpy as np
import csv  
from ultralytics import YOLO

# Load the pretrained YOLOv8 model
model = YOLO('fine_tuned_model(1).pt')

cap = cv2.VideoCapture('video2.mp4')

fps = int(cap.get(cv2.CAP_PROP_FPS)) 
frame_width = int(cap.get(3))  
frame_height = int(cap.get(4))  

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out = cv2.VideoWriter('processed_video2.avi', fourcc, fps, (frame_width, frame_height))

# Define the violation zone polygon coordinates
violation_zone_polygon = [np.array([[4, 1014], [807, 290], [1041, 287], [1758, 993]])]

def is_point_in_polygon(x, y, polygon):
    return cv2.pointPolygonTest(polygon[0], (float(x), float(y)), False) >= 0

def is_box_in_polygon(box, polygon, threshold=0.5):
    x1, y1, x2, y2 = map(int, box)
    bbox_points = np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]])
    points_inside = sum(is_point_in_polygon(x, y, polygon) for x, y in bbox_points)
    return points_inside / 4 >= threshold

video_duration = 42  # 
violation_time_start = video_duration - 17  
violation_time_end = video_duration  

violation_time_counter = 0  
violation_lane_counter = 0  
violated_trucks_time = set() 
violated_trucks_lane = set() 

bbox_shrink_factor = 0.8  

def get_violation_fine(query, violation_count, db):
    fine_result = db.similarity_search(query + " الغرامة")

    fine_details = fine_result[0].page_content if fine_result else "No fine details found"

    fine_description_match = re.search(r'^(.*?)[.]', fine_details)  
    penalty_amount_match = re.search(r'\d+[.,]?\d*', fine_details)  

    fine_description = fine_description_match.group(1) if fine_description_match else "No fine description found"
    penalty_amount = penalty_amount_match.group(0) if penalty_amount_match else "No penalty amount found"

    with open(f"fine_violation_{violation_count}.md", "w", encoding='utf-8') as file:
        file.write(f"{fine_details}\n\n")
        file.write(f"Fine description: {fine_description}\n")
        file.write(f"Penalty amount: {penalty_amount}\n")

    return fine_description, penalty_amount

with open('detection_logs.csv', mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Frame_Number', 'Timestamp', 'Object_ID', 'Class_ID', 'Confidence', 'BBox_X1', 'BBox_Y1', 'BBox_X2', 'BBox_Y2', 'Violation_Status']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    frame_number = 0  

    # Start video processing
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_number += 1  # Increment frame counter

        # Get the current video timestamp in seconds
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000

        # Check if current time is within violation time window
        is_in_violation_time = violation_time_start <= current_time <= violation_time_end

        # YOLOv8 detection and tracking
        results = model.track(source=frame, persist=True, verbose=False)

        # Draw violation zone polygon (lane)
        pts = np.array(violation_zone_polygon, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(frame, [pts], isClosed=True, color=(0, 0, 255), thickness=2)

        if is_in_violation_time:
            cv2.putText(frame, "Violation Time", (50, 300), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)

        for result in results:
            boxes = result.boxes
            for box in boxes:
                cls_id = int(box.cls[0])  # Class ID
                conf = float(box.conf[0])        # Confidence score
                if box.id is not None:
                    obj_id = int(box.id[0])       # Tracking ID
                else:
                    continue  # Skip if no tracking ID is available

                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)  # Bounding box coordinates

                # Check if detected object is a truck (adjust class ID based on your model)
                if cls_id == 3:  # Assuming class ID 3 is for trucks
                    violated = False  # Reset the violated flag for each object
                    violation_status = 'No Violation'  # Initialize violation status

                    # Calculate the center of the bounding box
                    x_center = (x1 + x2) / 2
                    y_center = (y1 + y2) / 2

                    # Calculate the width and height of the original bounding box
                    width = (x2 - x1) * bbox_shrink_factor
                    height = (y2 - y1) * bbox_shrink_factor

                    # Calculate new top-left and bottom-right corners based on shrink factor
                    new_x1 = x_center - width / 2
                    new_y1 = y_center - height / 2
                    new_x2 = x_center + width / 2
                    new_y2 = y_center + height / 2

                    shrunk_box = [new_x1, new_y1, new_x2, new_y2]

                    # Check for lane violation
                    if is_box_in_polygon(shrunk_box, violation_zone_polygon, threshold=0.5):
                        if obj_id not in violated_trucks_lane:
                            violation_lane_counter += 1
                            violated = True  # Mark as lane violation
                            violated_trucks_lane.add(obj_id)
                            violation_status = 'Lane Violation'

                            # Query RAG system for fine description and penalty amount
                            fine_description, penalty_amount = get_violation_fine(
                                "عدم التزام الشاحنات والمعدات الثقيلة بالسير في المسار الآمن في الطريق المتعدد المسارات. الغرامة المالية 3000 - 6000 ريال",
                                violation_lane_counter,
                                db
                            )
                            print(f"Extracted fine: {fine_description}, Extracted penalty: {penalty_amount}")
                        else:
                            violation_status = 'Lane Violation'

                    # Check for time violation
                    if is_in_violation_time:
                        if obj_id not in violated_trucks_time:
                            violation_time_counter += 1
                            violated = True  # Mark as time violation
                            violated_trucks_time.add(obj_id)
                            violation_status = 'Time Violation'

                            # Query RAG system for fine description and penalty amount
                            fine_description, penalty_amount = get_violation_fine(
                                "دخول الشاحنات والمعدات الثقيلة وما في حكمهما إلى المدن أو الخروج منها في الأوقات غير المسموح بها. الغرامة المالية 1000 - 2000 ريال",
                                violation_time_counter,
                                db
                            )
                            print(f"Extracted fine: {fine_description}, Extracted penalty: {penalty_amount}")
                        else:
                            if violation_status != 'Lane Violation':
                                violation_status = 'Time Violation'

                    # If both violations occurred
                    if obj_id in violated_trucks_lane and obj_id in violated_trucks_time:
                        violation_status = 'Lane and Time Violation'

                    # Draw bounding box (red for violating, green for compliant)
                    if violated:
                        color = (0, 0, 255)  # Red for violations
                    else:
                        color = (0, 255, 0)  # Green for no violations
                    cv2.rectangle(frame, (int(new_x1), int(new_y1)), (int(new_x2), int(new_y2)), color, 3)

                    # Put the object ID near the bounding box
                    cv2.putText(frame, f'ID: {obj_id}', (int(new_x1), int(new_y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 0.7, color, 2)

                    # **Log detection information to CSV**
                    writer.writerow({
                        'Frame_Number': frame_number,
                        'Timestamp': current_time,
                        'Object_ID': obj_id,
                        'Class_ID': cls_id,
                        'Confidence': conf,
                        'BBox_X1': int(new_x1),
                        'BBox_Y1': int(new_y1),
                        'BBox_X2': int(new_x2),
                        'BBox_Y2': int(new_y2),
                        'Violation_Status': violation_status
                    })

        # Display violation counts on the video
        cv2.putText(frame, f'Violations (Time): {violation_time_counter}', (50, 100), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2)
        cv2.putText(frame, f'Violations (Lane): {violation_lane_counter}', (50, 200), cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2)

        # Write frame to output video
        out.write(frame)

        # Optionally, display frame (for debugging)
        # cv2.imshow('Frame', frame)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


Extracted fine: دخول الشاحنات والمعدات الثقيلة وما في حكمهما إلى المدن أو الخروج منها في الأوقات غير المسموح بها, Extracted penalty: 1000
Extracted fine: دخول الشاحنات والمعدات الثقيلة وما في حكمهما إلى المدن أو الخروج منها في الأوقات غير المسموح بها, Extracted penalty: 1000
Extracted fine: دخول الشاحنات والمعدات الثقيلة وما في حكمهما إلى المدن أو الخروج منها في الأوقات غير المسموح بها, Extracted penalty: 1000
Extracted fine: دخول الشاحنات والمعدات الثقيلة وما في حكمهما إلى المدن أو الخروج منها في الأوقات غير المسموح بها, Extracted penalty: 1000


In [13]:
import os

# Save the API key
os.environ["GROQ_API_KEY"] = "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"

In [14]:
import os
from datetime import datetime
from twilio.rest import Client

# Writing fine description and penalty to the text file
def write_fine_to_file(fine_description, penalty_amount):
    with open("violation_fine.txt", "w", encoding="utf-8") as file:
        file.write(f"وصف المخالفة: {fine_description}\n")
        file.write(f"قيمة الغرامة: {penalty_amount} ريال\n")

# Read the RAG output from the text file and format the WhatsApp message
def format_whatsapp_message_from_file(id_number, city, vehicle):
    # Now open the file again in read mode
    with open("violation_fine.txt", "r", encoding="utf-8") as file:
        fine_details = file.read()  # Read the fine details from the file

    # Format the WhatsApp message
    message_template = f"""
تم قيد مخالفة:
{fine_details} - رصد آلي
رقم المخالفة: 123456789

رقم الهوية: {id_number}

التاريخ: {datetime.now().strftime("%Y-%m-%d")}

الوقت: {datetime.now().strftime("%H:%M:%S")}

المدينة: {city}

السيارة: {vehicle}

ويمكنكم الاعتراض خلال مدة أقصاها (30) يوماً من خلال منصة أبشر.
نأمل المبادرة بالسداد للاستفادة من تخفيض قيمة المخالفة بنسبة 25%
للاطلاع على تفاصيل المخالفة وسدادها يرجى زيارة منصة إيفاء: Efaa.sa ، كما يمكنك السداد من خلال القنوات الرسمية المعتمدة
"""
    return message_template

# WhatsApp message sending function
def send_whatsapp_message(account_sid, auth_token, from_whatsapp, to_whatsapp, message):
    client = Client(account_sid, auth_token)
    whatsapp_message = client.messages.create(
        body=message,
        from_=from_whatsapp,
        to=to_whatsapp
    )
    print(f"WhatsApp message sent to {to_whatsapp}: {whatsapp_message.sid}")
    return whatsapp_message.sid

# Example process combining everything
def main_process(account_sid, auth_token, from_whatsapp, to_whatsapp, id_number, city, vehicle, fine_description, penalty_amount):
    # Step 1: Write the fine details to a text file
    write_fine_to_file(fine_description, penalty_amount)

    # Step 2: Format the WhatsApp message by reading from the file
    message = format_whatsapp_message_from_file(id_number, city, vehicle)

    # Step 3: Send the WhatsApp message
    send_whatsapp_message(account_sid, auth_token, from_whatsapp, to_whatsapp, message)

# Twilio credentials and recipient info
account_sid = 'ACe154c002930bd94c255c7c7873983e7a'
auth_token = '93f34fc01b9c808b0a58750128901cf6'
from_whatsapp = 'whatsapp:+14155238886'
to_whatsapp = 'whatsapp:+966500489777'


main_process(account_sid, auth_token, from_whatsapp, to_whatsapp, "1234567890", "Riyadh", "Mercedes-Benz", fine_description, penalty_amount)

WhatsApp message sent to whatsapp:+966500489777: SMae90aaadff196075fa267527001a1b08
